In [ ]:
#####################################################################
#
#   aiplatform.CustomContainerTrainingJob
#
#####################################################################

# Create the training file/s and the dockerfile

In [ ]:
# create directory
! mkdir -p devdir/trainer

In [ ]:
#-------------------
# task.py
#-------------------

In [ ]:
%%writefile devdir/trainer/task.py

import os
import pickle
import argparse
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier

from google.cloud import storage

# parse args
parser = argparse.ArgumentParser()
parser.add_argument('--data_bucket_name', dest='data_bucket_name', default="", type=str, help='my gcs bucket with all training data -> w/o gs://')
parser.add_argument('--train_ds', dest='train_ds', default="", type=str, help='CSV file name of the training data')
args = parser.parse_args()


## set params
DATA_BUCKET_NAME = args.data_bucket_name
DATA_BUCKET_ROOT = f"/gcs/{DATA_BUCKET_NAME}"

DS_TRAIN_FILENAME = args.train_ds
DS_TRAIN_PATH = f"{DATA_BUCKET_ROOT}/{DS_TRAIN_FILENAME}"

MODEL_PATH = os.environ["AIP_MODEL_DIR"]
MODEL_SAVE_BUCKET = MODEL_PATH.replace('gs://','')
JOB_TYPE = os.environ["JOB_TYPE"]

#####################################################################
#
#   BEGIN
#
#####################################################################
# load data
df = pd.read_csv(DS_TRAIN_PATH)
labels = df.pop("label").tolist()
data = df.values.tolist()

# fit the model
skmodel = GradientBoostingClassifier(n_estimators = 100, max_depth = 10, min_samples_split = 100, min_samples_leaf = 100)
skmodel.fit(data, labels)

# save model
model_filename = "model.pkl"
with open(model_filename, 'wb') as model_file:
    pickle.dump(skmodel, model_file)

storage_client = storage.Client()
storage_path = os.path.join(MODEL_PATH, model_filename)
blob = storage.blob.Blob.from_string(storage_path, client=storage_client)
blob.upload_from_filename(model_filename)

#####################################################################
#
#   END
#
#####################################################################

In [ ]:
#-------------------
# Dockerfile
#-------------------

In [ ]:
%%writefile devdir/Dockerfile

# Specifies base image and tag
FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-8

WORKDIR /

# Installs additional packages
RUN pip install pip --upgrade

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

# Build the docker file and push to artifact registry

In [ ]:
from datetime import datetime
import os

In [ ]:
# specify parameters
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
P = ! gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_NUMBER)"
PROJECT_NUMBER = P[0]
REGION = "us-central1"
SERVICE_ACCOUNT = f"sa-vertex-pipelines@{PROJECT_ID}.iam.gserviceaccount.com"

# exercise details
USE_CASE = "CustomContainerTrainingJob"
ML_FRAMEWORK = "scikit"
MODEL_TYPE = "binclass"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

# repo details
REPO_FORMAT = "docker"
REPO_NAME = f"artifact-repo-{REPO_FORMAT}"
REPO_DESCRIPTION = f"'Description for {REPO_NAME}'"
IMAGE_URI = f"{REGION}-{REPO_FORMAT}.pkg.dev/{PROJECT_ID}/{REPO_NAME}/{ML_FRAMEWORK}:latest"

#-------------------
# GCS
#-------------------

# master source data
GCS_DATA_SOURCE_BUCKET_NAME = f"bkt-{REGION}-data"
GCS_DATA_SOURCE_BUCKET_PATH = f"gs://{GCS_DATA_SOURCE_BUCKET_NAME}"

# training data table
TRAIN_DS = "tab_class_10inps_1krows_tes_3498.csv"

# use case bucket
GCS_BUCKET_NAME = f"bkt-{REGION}-{USE_CASE.lower()}"
GCS_BUCKET_PATH = f"gs://{GCS_BUCKET_NAME}"
GCS_BUCKET_PATH_DATA = f"{GCS_BUCKET_PATH}/data"
GCS_BUCKET_PATH_CONFIGS = f"{GCS_BUCKET_PATH}/configs"
GCS_BUCKET_PATH_TMP = f"{GCS_BUCKET_PATH}/tmp"
GCS_BUCKET_PATH_STAGING = f"{GCS_BUCKET_PATH}/staging"

In [ ]:
# create use case gcs bucket if needed
! gsutil mb -p {PROJECT_ID} -c standard -l {REGION} {GCS_BUCKET_PATH}
! gsutil ls -L -b {GCS_BUCKET_PATH}

In [ ]:
# create the repo and configure
! gcloud artifacts repositories create {REPO_NAME} --repository-format={REPO_FORMAT} --location={REGION} --description={REPO_DESCRIPTION}
! gcloud auth configure-{REPO_FORMAT} {REGION}-{REPO_FORMAT}.pkg.dev

In [ ]:
# build and push
! docker build ./devdir/ -t {IMAGE_URI}
! docker push {IMAGE_URI}

In [ ]:
#! gcloud services enable containeranalysis.googleapis.com
! gcloud artifacts docker images describe {IMAGE_URI} --show-build-details

# Kick off CustomContainerTrainingJob

In [ ]:
from google.cloud import aiplatform

In [ ]:
# scit kit
TRAIN_VERSION  = "scikit-learn-cpu.0-23"
DEPLOY_VERSION = "sklearn-cpu.0-23"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket = GCS_BUCKET_PATH)

job = aiplatform.CustomContainerTrainingJob(
    display_name = f"CustomContainerTrainingJob_{TIMESTAMP}"
    , project = PROJECT_ID
    , location = REGION
    , staging_bucket = GCS_BUCKET_PATH
    , container_uri = IMAGE_URI
    , model_serving_container_image_uri = DEPLOY_IMAGE
)

In [ ]:
job.run(service_account = SERVICE_ACCOUNT
        , replica_count = 1
        , machine_type = "n1-standard-4"
        , model_display_name = "mymodelname"
        #, model_labels = { 'MY_KEY': 'MY_VALUE' }
        , args = [f"--data_bucket_name={GCS_DATA_SOURCE_BUCKET_NAME}", f"--train_ds={TRAIN_DS}"]
        , environment_variables = { 'JOB_TYPE': USE_CASE }
       )